# Model
This example is from Galvanin et al. 2007, *Model-Based Design of Parallel Experiments*.

$$
\begin{align}
    \frac{dx_1}{dt} &= (r - u_1 - \theta_4)x_1 \\
    \frac{dx_2}{dt} &= - \frac{rx_1}{\theta_3} + u_1(u_2 - x_2)\\
    r &= \frac{\theta_1 x_2}{\theta_2 + x_2}
\end{align}
$$

Where,  <br>
$x_1$ = biomass concentration ($g/L$). <br>
$x_2$ = substrate concentration ($g/L$).  <br>
$u_1$ = dilution factor ($h^{-1}$).  <br>
$u_2$ = substrate concentration in the feed ($g/L$).  <br>

**Experimental conditions** <br>
$x_1^0$ = initial biomass concentration ($1-10\; g/L$). <br>
$u_1 (t)$ = dilution factor ($0.05 - 0.20\; h^{-1}$).  <br>
$u_2 (t)$ = substrate concentration in the feed ($5-35\; g/L$).  <br>


**Measurements** <br>
$x_1 (t)$ = biomass concentration ($g/L$) <br>
$x_2 (t)$ = substrate concentration ($g/L$)  <br>

**Parameters** <br>
$\theta_1 (h^{-1}), \theta_2 (g/L), \theta_3 (\text{dimensionless}), \text{ \& } \theta_4 (h^{-1})$ <br>

**Other information** <br>
$x_2^0$ = initial substrate concentration ($0\; g/L$), cannot be manipulated. <br>
Total duration of the experiment, $\tau$ = $40\; h$ (fixed). <br>
Each experimental run involves 5 sampling times. <br>
Inputs $\mathbf{u}(t)$ can be manipulated and represented as a piece-wise constant profile over 5 switching intervals. <br>
The sampling times and the control variables switching times can be different. <br>
The elapsed time between any two sampling points is allowed to be between 0.1 and 20 h, 
and the duration of each control interval is allowed to be between 0.2 and 20 h. <br>

# Model with True parameters

In [5]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar, Simulator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [47]:
# True parameter values for the mechanistic model
theta_true = [0.310, 0.180, 0.550, 0.050]
def create_biomass_model(time_horizon=40, initials={"x_10": 10, "u1": 0.20, "u2": 35, "s_time": 20}):
    """Creates a Pyomo model for biomass growth.

    Parameters
    ----------
    time_horizon : int, optional
        Total duration of the experiment, by default 40
    initials : dict, optional
        Initial values for model variables, by default {"x_10": 10, "u1": 0.20, "u2": 35, "s_time": 20}
    """
    m = pyo.ConcreteModel()

    # Times Set
    m.t = ContinuousSet(bounds=(0, time_horizon))

    # Parameters
    m.theta1 = pyo.Var(initialize=1, domain=pyo.PositiveReals)
    m.theta2 = pyo.Var(initialize=1, domain=pyo.PositiveReals)
    m.theta3 = pyo.Var(initialize=1, domain=pyo.PositiveReals)
    m.theta4 = pyo.Var(initialize=1, domain=pyo.PositiveReals)

    # Time-varying inputs
    m.u1 = pyo.Var(m.t, initialize=initials["u1"])
    m.u2 = pyo.Var(m.t, initialize=initials["u2"])

    # State Variables
    m.x1 = pyo.Var(m.t, initialize=initials["x_10"], domain=pyo.NonNegativeReals)
    m.x2 = pyo.Var(m.t, initialize=0, domain=pyo.NonNegativeReals)

    # Derivatives
    m.dx1dt = DerivativeVar(m.x1, wrt=m.t)
    m.dx2dt = DerivativeVar(m.x2, wrt=m.t)

    # r expression
    def _r_rule(m, t):
        return m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
    m.r = pyo.Expression(m.t, rule=_r_rule)

    # ODEs
    def _dx1dt_rule(m, t):
        return m.dx1dt[t] == (m.r[t] - m.u1[t] -m.theta4) * m.x1[t]
    m.dx1dt_con = pyo.Constraint(m.t, rule=_dx1dt_rule)


    def _dx2dt_rule(m, t):
        return m.dx2dt[t] == - m.r[t] * m.x1[t] / m.theta3 + m.u1[t] * (m.u2[t] - m.x2[t])
    m.dx2dt_con = pyo.Constraint(m.t, rule=_dx2dt_rule)
    return m\
    


### --------------------------------------------------
def generate_synthetic_data(initial_x1, control_profile, noise_matrix="A"):
    """Generates synthetic data for biomass growth model.

    Parameters
    ----------
    initial_x1 : float
        Initial biomass concentration.
    """
    # Model
    model = create_biomass_model(time_horizon=40)

    ### Set true parameter values
    model.theta1.fix(theta_true[0])
    model.theta2.fix(theta_true[1])
    model.theta3.fix(theta_true[2])
    model.theta4.fix(theta_true[3])

    ### Set initial condition
    model.x1[0].fix(initial_x1)
    model.x2[0].fix(0.0)

    ### Discretize Inputs
    tsim = np.linspace(0, 40, 401)  # 401 time points = 0.1h step size

    # Apply control profile function to finx u1/u2 at specified time points
    for t in tsim:
        if t not in model.t:
            model.t.add(t)  # Add simulation time point to ContinuousSet

    
    ### Create a desnse input profiles
    # We need to map every t in tsim to a control value based on control_profile
    u1_dense = {}
    u2_dense = {}
    sorted_control_times = sorted(control_profile.keys())
    for t in tsim:
        active_t =0
        for ct in sorted_control_times:
            if ct <= t:
                active_t = ct
            else:
                break
        u1_dense[t] = control_profile[active_t][0]
        u2_dense[t] = control_profile[active_t][1]

    ### Initialize Simulator
    sim = Simulator(model, package='scipy')  

    ### Apply controls before simulation
    # We must ensure u1/u2 are fixed at every time point in the integrator
    # For a real dynamic simulator, we often need a callback, but Pyomo
    # simulator can handle time-indexed vars if they are intialized/fixed

    ### Simulate
    # Varying inputs Suffix
    varying_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
    varying_inputs[model.u1] = u1_dense
    varying_inputs[model.u2] = u2_dense
    results = sim.simulate(
        numpoints=len(tsim),
        integrator='vode',
        varying_inputs=varying_inputs,
    )

    # Convert to DataFrame
    df = pd.DataFrame(results, columns=["t", "x1", "x2"])

    ### Select Measurement Points
    # Paper mentioned 5 sample times per experiment
    # For data generation, we can just take the specfied time point we want
    # Let's assume the following sampling times
    sample_times = [1, 5, 15, 20, 30]
    df_measured = df[df["t"].isin(sample_times)].copy()

    ### Add Measurement Noise
    # Sigma_A = diag(0.01, 0.05)
    # Sigma_B = diag(0.1, 0.5)

    std_devs = {}
    if noise_matrix == "A":
        std_devs = {"x1": np.sqrt(0.01), "x2": np.sqrt(0.05)}
    elif noise_matrix == "B":
        std_devs = {"x1": np.sqrt(0.1), "x2": np.sqrt(0.5)}
    else:
        raise ValueError("Invalid noise matrix type. Choose 'A' or 'B'.")

    np.random.seed(42)  # For reproducibility
    df_measured["x1_noisy"] = df_measured["x1"] + np.random.normal(
        0, std_devs["x1"], size=len(df_measured)
    )
    df_measured["x2_noisy"] = df_measured["x2"] + np.random.normal(
        0, std_devs["x2"], size=len(df_measured)
    )

    return df, df_measured


### --------------------------------------------------
# ---- Example Usage ----
# Define a piecewise constant control profile
# Format: {time: (u1, u2)}
# 0-10h:  u1=0.15, u2=30.0
# 10-25h: u1=0.20, u2=10.0
# 25-40h: u1=0.05, u2=20.0
controls_dict = {0: (0.15, 30.0), 10: (0.20, 10.0), 25: (0.05, 20.0)}

# Generate synthetic data
true_data, measured_data = generate_synthetic_data(
    initial_x1=5, control_profile=controls_dict
)

DAE_Error: When simulating with Scipy you must provide values for all parameters and algebraic variables that are indexed by the ContinuoutSet using the 'varying_inputs' keyword argument. Please refer to the simulator documentation for more information.

In [52]:
def create_biomass_model(
    time_horizon=40, initials={"x_10": 10, "u1": 0.20, "u2": 35, "s_time": 20}
):
    """Creates a Pyomo model for biomass growth."""
    m = pyo.ConcreteModel()

    # Times Set
    m.t = ContinuousSet(bounds=(0, time_horizon))

    # Parameters
    m.theta1 = pyo.Var(initialize=1, domain=pyo.PositiveReals)
    m.theta2 = pyo.Var(initialize=1, domain=pyo.PositiveReals)
    m.theta3 = pyo.Var(initialize=1, domain=pyo.PositiveReals)
    m.theta4 = pyo.Var(initialize=1, domain=pyo.PositiveReals)

    # Time-varying inputs as Params
    m.u1 = pyo.Param(m.t, initialize=initials["u1"], mutable=True)
    m.u2 = pyo.Param(m.t, initialize=initials["u2"], mutable=True)

    # State Variables
    m.x1 = pyo.Var(m.t, initialize=initials["x_10"], domain=pyo.NonNegativeReals)
    m.x2 = pyo.Var(m.t, initialize=0, domain=pyo.NonNegativeReals)

    # Derivatives
    m.dx1dt = DerivativeVar(m.x1, wrt=m.t)
    m.dx2dt = DerivativeVar(m.x2, wrt=m.t)

    # REMOVED: m.r Expression - inline it instead

    # ODEs - with r inlined
    def _dx1dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx1dt[t] == (r - m.u1[t] - m.theta4) * m.x1[t]

    m.dx1dt_con = pyo.Constraint(m.t, rule=_dx1dt_rule)

    def _dx2dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx2dt[t] == -r * m.x1[t] / m.theta3 + m.u1[t] * (m.u2[t] - m.x2[t])

    m.dx2dt_con = pyo.Constraint(m.t, rule=_dx2dt_rule)

    return m

In [69]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar
from pyomo.dae.simulator import Simulator
import numpy as np
import pandas as pd

# True parameter values for the mechanistic model
theta_true = [0.310, 0.180, 0.550, 0.050]


def create_biomass_model(
    time_horizon=40, theta_initial=None
):
    """Creates a Pyomo model for biomass growth."""
    m = pyo.ConcreteModel()

    # Times Set
    m.t = ContinuousSet(bounds=(0, time_horizon))

    # Parameters (fixed during simulation)
    m.theta1 = pyo.Var(initialize=theta_initial[0] if theta_initial else 1, domain=pyo.PositiveReals)
    m.theta2 = pyo.Var(initialize=theta_initial[1] if theta_initial else 1, domain=pyo.PositiveReals)
    m.theta3 = pyo.Var(initialize=theta_initial[2] if theta_initial else 1, domain=pyo.PositiveReals)
    m.theta4 = pyo.Var(initialize=theta_initial[3] if theta_initial else 1, domain=pyo.PositiveReals)

    # Time-varying inputs
    m.u1 = pyo.Var(m.t)
    m.u2 = pyo.Var(m.t)

    # State Variables
    m.x1 = pyo.Var(m.t, domain=pyo.NonNegativeReals)
    m.x2 = pyo.Var(m.t, domain=pyo.NonNegativeReals)

    # Derivatives
    m.dx1dt = DerivativeVar(m.x1, wrt=m.t)
    m.dx2dt = DerivativeVar(m.x2, wrt=m.t)

    # ODEs - with r inlined to avoid Expression evaluation issues
    def _dx1dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx1dt[t] == (r - m.u1[t] - m.theta4) * m.x1[t]

    m.dx1dt_con = pyo.Constraint(m.t, rule=_dx1dt_rule)

    def _dx2dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx2dt[t] == -r * m.x1[t] / m.theta3 + m.u1[t] * (m.u2[t] - m.x2[t])

    m.dx2dt_con = pyo.Constraint(m.t, rule=_dx2dt_rule)

    return m


def generate_synthetic_data(initial_x1, u1_profile, u2_profile, noise_matrix="A"):
    """Generates synthetic data for biomass growth model."""
    model = create_biomass_model(time_horizon=40)

    # Set true parameter values
    model.theta1.fix(theta_true[0])
    model.theta2.fix(theta_true[1])
    model.theta3.fix(theta_true[2])
    model.theta4.fix(theta_true[3])

    # Set initial condition
    model.x1[0].fix(initial_x1)
    model.x2[0].fix(0.0)

    # Initialize Simulator
    sim = Simulator(model, package='scipy')

    # Create Suffix for varying inputs
    varying_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
    varying_inputs[model.u1] = u1_profile
    varying_inputs[model.u2] = u2_profile

    # Simulate - returns (time_array, profiles_array)
    tsim = np.linspace(0, 40, 401)
    tsim_result, profiles = sim.simulate(
        numpoints=len(tsim),
        integrator='vode',
        varying_inputs=varying_inputs,
    )

    # Create DataFrame from time and profiles
    df = pd.DataFrame({'t': tsim_result, 'x1': profiles[:, 0], 'x2': profiles[:, 1]})

    # Select Measurement Points
    sample_times = [1, 5, 15, 20, 30]
    df_measured = df[df["t"].isin(sample_times)].copy()

    # Add Measurement Noise
    std_devs = {}
    if noise_matrix == "A":
        std_devs = {"x1": np.sqrt(0.01), "x2": np.sqrt(0.05)}
    elif noise_matrix == "B":
        std_devs = {"x1": np.sqrt(0.1), "x2": np.sqrt(0.5)}
    else:
        raise ValueError("Invalid noise matrix type. Choose 'A' or 'B'.")

    np.random.seed(42)
    df_measured["x1_noisy"] = df_measured["x1"] + np.random.normal(
        0, std_devs["x1"], size=len(df_measured)
    )
    df_measured["x2_noisy"] = df_measured["x2"] + np.random.normal(
        0, std_devs["x2"], size=len(df_measured)
    )

    return df, df_measured


# Example usage
controls_dict = {0: (0.15, 30.0), 10: (0.20, 10.0), 25: (0.05, 20.0)}
u1_profile = {t: vals[0] for t, vals in controls_dict.items()}
u2_profile = {t: vals[1] for t, vals in controls_dict.items()}
true_data, measured_data = generate_synthetic_data(
    initial_x1=5, u1_profile=u1_profile, u2_profile=u2_profile
)

In [71]:
true_data

,t,x1,x2
0,0.0,5.000000,0.000000
1,0.1,4.969858,0.321184
2,0.2,4.979694,0.565755
3,0.3,5.001896,0.783783
4,0.4,5.030735,0.985666
...,...,...,...
396,39.6,5.259586,0.091078
397,39.7,5.261761,0.091020
398,39.8,5.263913,0.090964
399,39.9,5.266044,0.090908


In [63]:
measured_data

,t,x1,x2,x1_noisy,x2_noisy
10,1.0,5.265617,2.000276,5.315288,1.947921
50,5.0,7.694514,4.118824,7.680688,4.471947
150,15.0,6.391754,0.206251,6.456523,0.377854
200,20.0,4.804952,0.406633,4.957255,0.301655
300,30.0,4.905243,0.101489,4.881827,0.222809


# Create the Experiment Class

In [59]:
from pyomo.contrib.parmest.experiment import Experiment
from pyomo.contrib.parmest import parmest
from pyomo.contrib.doe import DesignOfExperiments


In [ ]:
class BiomassExperiment(Experiment):
    def __init__(self, data_df, theta_initial, u1_profile, u2_profile):
        """Biomass Experiment class for parameter estimation and design of experiments.

        Parameters
        ----------
        data_df : pandas.DataFrame
            The measured data for the experiment.
        theta_initial : list
            Initial guess for the parameters [theta1, theta2, theta3, theta4].
        u1_profile : dict
            The profile for u1 (dilution factor).
        u2_profile : dict
            The profile for u2 (substrate concentration in feed).
        """
        self.data = data_df
        self.theta_initial = theta_initial
        self.u1_profile = u1_profile
        self.u2_profile = u2_profile
        self.model = None

    def create_model(self):
        self.model = create_biomass_model(theta_initial=self.theta_initial)
        return self.model
    
    def _apply_inputs_to_model(self, model):
        """
        Helper function to interpolate and fix inputs u1/u2 at all discretized time points
        """
        # Sort keys for zero-oder hold logic
        u1_times = sorted(self.u1_profile.keys())
        u2_times = sorted(self.u2_profile.keys())

        def get_val(t, times, profile):
            active_t = 0
            for ct in times:
                if ct <= t:
                    active_t = ct
                else:
                    break
            return profile[active_t]
        
        # Iterate over all time points in discretized model
        for t in model.t:
            model.u1[t].fix(get_val(t, u1_times, self.u1_profile))
            model.u2[t].fix(get_val(t, u2_times, self.u2_profile))
    
    def finalize_model(self):
        m = self.model
        # Fix the parameter values
        m.theta1.fix()
        m.theta2.fix()
        m.theta3.fix()
        m.theta4.fix()

        ### Add critical time points before discretization
        # Add Measurement times
        if self.data is not None:
            measure_times = self.data["t"].unique()
            for t in measure_times:
                if t not in m.t:
                    m.t.add(t)
        # Add control switch times
        control_times = list(self.u1_profile.keys()) + list(self.u2_profile.keys())
        control_times = sorted(set(control_times))
        for t in control_times:
            if t not in m.t and t <= m.t.upper:
                m.t.add(t)
       
        ### Discretize the model
        discretizer = pyo.TransformationFactory('dae.collocation')
        discretizer.apply_to(m, nfe=20, ncp=3, scheme='LAGRANGE-RADAU')

        ### Fix inputs at all discretized time points
        self._apply_inputs_to_model(m)

        #


    
        

In [ ]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar
import pandas as pd
import numpy as np
import pyomo.contrib.parmest.parmest as parmest

# Global true parameters (for reference or synthetic data generation)
theta_true = [0.310, 0.180, 0.550, 0.050]


def create_biomass_model(time_horizon=40, initials=None, theta_initial=None):
    """
    Creates the base Pyomo model.
    """
    if initials is None:
        initials = {"x_10": 10, "u1": 0.20, "u2": 35}

    m = pyo.ConcreteModel()

    # Times Set
    m.t = ContinuousSet(bounds=(0, time_horizon))

    # Parameters to be estimated
    # We allow them to be variables for estimation, but initialize them
    val_t1 = theta_initial[0] if theta_initial else 1.0
    val_t2 = theta_initial[1] if theta_initial else 1.0
    val_t3 = theta_initial[2] if theta_initial else 1.0
    val_t4 = theta_initial[3] if theta_initial else 1.0

    m.theta1 = pyo.Var(initialize=val_t1, domain=pyo.PositiveReals)
    m.theta2 = pyo.Var(initialize=val_t2, domain=pyo.PositiveReals)
    m.theta3 = pyo.Var(initialize=val_t3, domain=pyo.PositiveReals)
    m.theta4 = pyo.Var(initialize=val_t4, domain=pyo.PositiveReals)

    # Time-varying inputs (Variables in Pyomo.DAE)
    # We initialize them, but they will be fixed via profiles later
    m.u1 = pyo.Var(m.t, initialize=initials["u1"])
    m.u2 = pyo.Var(m.t, initialize=initials["u2"])

    # State Variables
    m.x1 = pyo.Var(m.t, initialize=initials["x_10"], domain=pyo.NonNegativeReals)
    m.x2 = pyo.Var(m.t, initialize=0, domain=pyo.NonNegativeReals)

    # Derivatives
    m.dx1dt = DerivativeVar(m.x1, wrt=m.t)
    m.dx2dt = DerivativeVar(m.x2, wrt=m.t)

    # ODEs
    def _dx1dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx1dt[t] == (r - m.u1[t] - m.theta4) * m.x1[t]

    m.dx1dt_con = pyo.Constraint(m.t, rule=_dx1dt_rule)

    def _dx2dt_rule(m, t):
        r = m.theta1 * m.x2[t] / (m.theta2 + m.x2[t])
        return m.dx2dt[t] == -r * m.x1[t] / m.theta3 + m.u1[t] * (m.u2[t] - m.x2[t])

    m.dx2dt_con = pyo.Constraint(m.t, rule=_dx2dt_rule)

    return m


class BiomassExperiment:
    def __init__(self, data_df, theta_guess, u1_profile, u2_profile):
        """
        Class to handle Model Creation, Discretization, and Parameter Estimation.

        Parameters
        ----------
        data_df : pd.DataFrame
            Contains columns ['t', 'x1_noisy', 'x2_noisy'] (or similar)
        theta_guess : list
            Initial guess for [theta1, theta2, theta3, theta4]
        u1_profile, u2_profile : dict
            Input profiles {time: value}
        """
        self.data = data_df
        self.theta_initial = theta_guess
        self.u1_profile = u1_profile
        self.u2_profile = u2_profile
        self.model = None

    def _apply_inputs_to_model(self, model):
        """
        Helper to interpolate and fix inputs u1/u2 at ALL discretized time points.
        This is crucial for Collocation where intermediate points exist.
        """
        # Sort keys for zero-order hold logic
        u1_times = sorted(self.u1_profile.keys())
        u2_times = sorted(self.u2_profile.keys())

        def get_val(t, times, profile):
            # Zero-order hold interpolation
            active_t = times[0]
            for ct in times:
                if ct <= t:
                    active_t = ct
                else:
                    break
            return profile[active_t]

        # Iterate over ALL time points in the discretized model
        for t in model.t:
            val_u1 = get_val(t, u1_times, self.u1_profile)
            val_u2 = get_val(t, u2_times, self.u2_profile)

            model.u1[t].fix(val_u1)
            model.u2[t].fix(val_u2)

    def create_discretized_model(self):
        """
        Creates the model, adds data points to time set, and discretizes.
        This is the format required by Parmest.
        """
        # 1. Create Base Model
        m = create_biomass_model(theta_initial=self.theta_initial)

        # 2. Add Measurement Times to ContinuousSet BEFORE Discretization
        # This ensures we have variables exactly at the measurement times
        if self.data is not None:
            measure_times = self.data['t'].unique()
            for t in measure_times:
                if t not in m.t:
                    m.t.add(t)

        # 3. Discretize (Orthogonal Collocation)
        # nfe=10 means 10 finite elements. ncp=3 is standard for Radau.
        discretizer = pyo.TransformationFactory('dae.collocation')
        discretizer.apply_to(m, nfe=20, ncp=3, scheme='LAGRANGE-RADAU')

        # 4. Fix Inputs at all grid points
        self._apply_inputs_to_model(m)

        # 5. Fix Initial Conditions (Time 0)
        # Assuming initial conditions are in the first row of data or known
        m.x1[0].fix(5.0)  # Or take from self.data
        m.x2[0].fix(0.0)

        self.model = m
        return m

    def define_objective(self, model):
        """
        Define the Sum of Squared Errors (SSE) objective for parmest.
        """
        expr = 0
        # Loop over data and add error terms
        for idx, row in self.data.iterrows():
            t = row['t']
            # Only add error term if t exists in the model (it should, due to step 2 above)
            if t in model.t:
                # Weighted SSE (simple version)
                expr += (model.x1[t] - row['x1_noisy']) ** 2
                expr += (model.x2[t] - row['x2_noisy']) ** 2

        model.obj = pyo.Objective(expr=expr, sense=pyo.minimize)

    def run_parameter_estimation(self):
        """
        Runs parmest using the discretized model.
        """

        # Define the function parmest will call to get a fresh model
        def parmest_model_function():
            m = self.create_discretized_model()
            self.define_objective(m)
            return m

        # Variables to estimate
        theta_names = ['theta1', 'theta2', 'theta3', 'theta4']

        # Setup Parmest Estimator
        # Note: We don't pass 'data' to create_estimator here in the standard list format
        # because we manually built the objective function inside the model.
        # This is often more robust for DAEs with specific time points.
        pest = parmest.Estimator(
            parmest_model_function, data=None, theta_names=theta_names
        )

        # Run optimization
        # You need a nonlinear solver installed (e.g., ipopt)
        obj_val, theta_est = pest.theta_est(solver='ipopt')

        return theta_est


# --------------------------------------------------------------------
# Example Workflow
# --------------------------------------------------------------------

if __name__ == "__main__":
    # 1. Setup Dummy Data (Using the synthetic generation logic you have)
    # For demonstration, creating a small dataframe manually
    data_points = {
        't': [1.0, 5.0, 15.0, 20.0, 30.0],
        'x1_noisy': [4.5, 6.2, 12.1, 14.5, 18.2],  # Fake data
        'x2_noisy': [0.5, 3.2, 8.5, 5.1, 2.0],  # Fake data
    }
    df_data = pd.DataFrame(data_points)

    # 2. Define Input Profiles
    controls_dict = {0: (0.15, 30.0), 10: (0.20, 10.0), 25: (0.05, 20.0)}
    u1_prof = {t: vals[0] for t, vals in controls_dict.items()}
    u2_prof = {t: vals[1] for t, vals in controls_dict.items()}

    # 3. Initialize Experiment
    # Giving a slightly wrong guess to see if it optimizes
    initial_guess = [0.5, 0.5, 0.5, 0.5]

    experiment = BiomassExperiment(df_data, initial_guess, u1_prof, u2_prof)

    # 4. Run Estimation (Requires Ipopt)
    try:
        print("Starting Parameter Estimation...")
        # Note: This will fail if 'ipopt' is not installed in your environment
        estimated_params = experiment.run_parameter_estimation()
        print("\nEstimated Parameters:")
        print(estimated_params)
    except Exception as e:
        print(f"\nSolver error (expected if ipopt is missing): {e}")
        print("To run this, ensure you have the Ipopt solver installed.")